# Python 101
## XI. Webscraping - requests + Beautifulsoup

---

### 0. Easy file sharing
Start your own web-server:
- in command prompt change your directory to `notebooks/lectures/python_beginner/data`
- start the server with the `python -m http.server` command

### 1. Obtain a webpage

The easiest way is to use a third party library called __`requests`__. Let's import it right away!

In [ ]:
import requests

And then we simply ask a server to give us an html document by requesting it through an url.

In [ ]:
existing_url = 'http://localhost:8000/test.html'
response = requests.get(existing_url)
print(response.status_code) # hopefully 200 -> successful download

In [ ]:
not_existing_url = 'http://fulibacsi.github.io/data/test1.html'
response = requests.get(not_existing_url)
print(response.status_code) # unfortunately 404 -> not exists

__Common status codes:__
- 200: success
- 301: permanent redirect
- 303: redirect
- 400: bad request
- 401: unauthorized
- 404: not exists

In [ ]:
response = requests.get(existing_url)
print(response.content)

Jupyter can render the page if it was successfully downloaded.

In [ ]:
from IPython.display import HTML
if response.status_code == 200:
    result = HTML(response.content)
else:
    result = 'Nah, let\'s have a beer instead!'

In [ ]:
result

### 2. Let's parse it!

We have a third party module for this purpose as well, the __`BeautifulSoup`__.  
Let's import it!

In [ ]:
from bs4 import BeautifulSoup

Then create a soup from the downloaded document.

In [ ]:
document = response.content
soup = BeautifulSoup(document, 'html.parser')

In [ ]:
print(soup.prettify())

With the created soup (which is a parsed document) we can easily access any part of the document.  
Let's try to:
- get the title of the document

In [ ]:
print(soup.title)

- get the title text

In [ ]:
print(soup.title.getText())

- get the text-only version of the page

In [ ]:
print(soup.getText())

- get all the links from the document

In [ ]:
soup.findAll('a')

- get the actual urls from the tags

In [ ]:
for url in soup.findAll('a'):
    print(url.get('href'))

During scraping, there are a lot of different tasks that must be solve in order to get the data we need. 
In this case this demo document has important and unimportant parts. We only need the important parts.   
#### a) Let's find the important links!

In [ ]:
important_urls = []
for url in soup.findAll('a'):
    if 'important_part' in url.get('href'):
        important_urls.append(url.get('href'))
print(important_urls)

#### b) Let's find the important text in the document
- select every paragraph which has "important" class

In [ ]:
soup.findAll('p', {'class': 'important'})

- Whooops, something's going on! Let's investigate!

In [ ]:
important_paragraphs = soup.findAll('p', {'class': 'important'})

- print the text in the tags, and tags' parent's id attribute

In [ ]:
for p in important_paragraphs:
    print(p.getText(), p.parent.get('id'))

- We can see, that the "fake" result is from somewhere else

In [ ]:
print(soup.find(id='not_main_section'))

- We have a hidden fake section! Let's modify our search!

In [ ]:
soup.find(id='main_content').findAll('p', {'class': 'important'})

#### c) Let's find the pictures of our interest
- Let's have the "nice" pictures from the div with random_images_1 class!

In [ ]:
(
    soup
    .find(id='main_content')
    .find('div', {'class': 'random_images_1'})
    .findAll('img', {'class': 'nice'})
)

- Whoops again. Filter out the result we don't like.

In [ ]:
imgs = (
    soup
    .find(id='main_content')
    .find('div', {'class': 'random_images_1'})
    .findAll('img', {'class': 'nice'})
)
nice_imgs = []
for img in imgs:
    if 'not' not in img['class']:
        nice_imgs.append(img['src'])
print(nice_imgs)

## Exercise

#### 1. Save every important link to a file

In [ ]:
BASE_URI = './data/'
filename = 'important_urls.txt'
# your code goes here

#### 2. Let's get a random img/gif url from 9gag!
- get the page from http://9gag.com/random
- find the images, and get the `src` attribute's value
- depending on the content you can find:
    - not animated images are stored in __`img`__ tags with __`badge-item-img`__ class, in the __`src`__ attribute 
    - animated imgages are stored in __`div`__ tags with __`badge-animated-container-animated`__ class, in the __`data-mp4`__ attribute

In [ ]:
URI = "http://9gag.com/random"
# your code goes here

---

### 3. Querying webpages 

Collect the articles about migrants from index.hu

This will require to search in the site.
On the upper-left corner, there is a search icon. Use it, and observe the resulting url:

`http://index.hu/24ora/?tol=1999-01-01&ig=2017-11-07&s=migráns`

It has multiple parts:
- `http://` - protocol
- `index.hu` - base url
- `/24ora/` - sub url
- `?tol=1999-01-01&ig=2017-11-07&s=migráns` - query

Let's investigate the query part a little more!  
Every query starts with a __`?`__ charater followed by one or more key-value pairs. The key-value pairs are separated with the __`&`__ character. Based on this information, we can extract the query parameters:
- `tol`
- `ig`
- `s`

Use these values to construct our own request:

In [ ]:
base_url = 'http://index.hu'
sub_url = '/24ora'
query = {
    'tol': '1999-01-01',
    'ig': '2017-11-07',
    's': u'migráns'
}

We can use the requests library to send the query:

In [ ]:
resp = requests.get(url=base_url+sub_url, data=query) # some pages requires `params` instead of `data`
resp

Using the response, extract the urls inside the `<article>` tags!

You can see that only 30 results showed up. We can customize our query to cover shorter amount of timed by replacing __`tol`__ and __`ig`__ parameters with a formattable string: __`'{year}-{month:0>2}-{day:0>2}'`__. This string can be formatted by providing the required parameters:
- year
- month
- day

like this:

In [ ]:
'{year}-{month:0>2}-{day:0>2}'.format(year=2016, month=1, day=1)

There is a useful library called __`datetime`__. You can use it to generate dates automatically.

In [ ]:
import datetime

date = datetime.date(1999, 1, 1)
day_after_date = date + datetime.timedelta(days=1)
day_before_date = date - datetime.timedelta(days=1)
today = datetime.date.today()

print(day_before_date)
print(date)
print(day_after_date)
print(today)

print(today.year, today.month, today.day)

Create a loop which iterate through every day from 1999-01-01 till today and execute the same procedure you created previously. (Pro tip: create a function!) Observe the number of results!

---

### 4. User agents

Let's pretend to be a browser instead of a script

In [ ]:
USER_AGENTS = [
    # Chrome
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36',
    # Firefox
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:25.0) Gecko/20100101 Firefox/25.0',
    # Opera
    'Opera/9.80 (Windows NT 6.0) Presto/2.12.388 Version/12.14',
    # Safari
    'Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25',
    # Internet Explorer, probably a good idea to leave this one out...
    'Mozilla/5.0 (compatible; MSIE 10.6; Windows NT 6.1; Trident/5.0; InfoPath.2; SLCC1; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; .NET CLR 2.0.50727) 3gpp-gba UNTRUSTED/1.0',
]

Let's write a wrapper function to handle the user-agent string.

In [ ]:
import random
def get_header(agents):
    return {'User-agent': random.choice(agents)}

### Exercise

#### 3. Get the main articles from index.hu
Write a function that prints that extracts the current main articles! It should contain:
- the title
- the article text
- the url
- every picture from the article

In [ ]:
url = 'http://index.hu'
index_response = requests.get(url, headers=get_header(USER_AGENTS))

#### 4. Get the articles about migration from 444.hu

Write a function that prints the titles of the articles

In [ ]:
url = 'https://444.hu/kereses'
query = '?q=migrans'